In [ ]:
## Importing libraries

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import warnings
warnings.filterwarnings('ignore')

SEED = 42

Mounted at /content/drive


In [ ]:
##### The following codes were inspired by the following kernels:
######### 1/ Raghuram Nandepu ("https://medium.com/analytics-vidhya/santander-customer-transaction-prediction-an-end-to-end-machine-learning-project-2cb763172f8a") - primarily the part on oversampling and generation of the model

## FE1

In [ ]:
## Importing data

df_train = pd.read_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-sum/train_fe1_sum_lgbm.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe1-sum/test_fe1_sum_lgbm.csv")

In [ ]:
## Oversampling 

def oversampling(x, y, t=2):
    
    xs, xn = [], []
    
    for i in range(t // 2):
        mask = y == 0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        featnum = x1.shape[1] // 200 - 1

        for c in range(200):
            np.random.shuffle(ids)
            x1[:, [c] + [200 + featnum * c + idc for idc in range(featnum)]] = x1[ids][:, [c] + [200 + featnum * c + idc for idc in range(featnum)]]
        xn.append(x1)
    
    for i in range(t):
        mask = y > 0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        featnum = x1.shape[1] // 200 - 1
        
        for c in range(200):
            np.random.shuffle(ids)
            x1[:, [c] + [200 + featnum * c + idc for idc in range(1)]] = x1[ids][:, [c] + [200 + featnum * c + idc for idc in range(1)]]
        xs.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x, xs, xn])
    y = np.concatenate([y, ys, yn])
    
    return x, y

In [ ]:
gbdt_param1 = {
    # Core Parameters
    'objective': 'binary',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 15,
    'tree_learner': 'serial',
    'num_threads': 8,
    'seed': SEED,
    
    # Learning Control Parameters
    'max_depth': -1,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 10,  
    'bagging_fraction': 0.6,
    'bagging_freq': 5,
    'feature_fraction': 0.05,
    'lambda_l1': 1.,
    'bagging_seed': SEED,
    
    # Others
    'verbosity ': 1,
    'boost_from_average': False,
    'metric': 'auc',
}

In [ ]:
predictors = df_train.columns.tolist()[2:]
X_test = df_test[predictors]

n_splits = 5
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

predictions = df_test[['ID_code']]

In [ ]:
for fold, (train_ind, val_ind) in enumerate(skf.split(df_train, df_train.target.values)):
    
    X_train, y_train = df_train.iloc[train_ind][predictors], df_train.iloc[train_ind]['target']
    X_valid, y_valid = df_train.iloc[val_ind][predictors], df_train.iloc[val_ind]['target']

    N = 1
    p_valid, yp = 0, 0
        
    for i in range(N):
        print('\nFold {} - N {}'.format(fold + 1, i + 1))
        
        X_t, y_t = oversampling(X_train.values, y_train.values)
        weights = np.array([0.8] * X_t.shape[0])
        weights[:X_train.shape[0]] = 1.0
        print('Shape of X_train after augment: {}\nShape of y_train after augment: {}'.format(X_t.shape, y_t.shape))
        
        X_t = pd.DataFrame(X_t)
        X_t = X_t.add_prefix('var_')
    
        trn_data = lgb.Dataset(X_t, label=y_t, weight=weights)
        val_data = lgb.Dataset(X_valid, label=y_valid)
        evals_result = {}
        
        lgb_clf = lgb.train(gbdt_param1, trn_data, 100000, valid_sets=[trn_data, val_data], early_stopping_rounds=5000, verbose_eval=1000, evals_result=evals_result)
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
        

    predictions['fold{}'.format(fold + 1)] = yp / N


Fold 1 - N 1
Shape of X_train after augment: (336079, 1000)
Shape of y_train after augment: (336079,)
Training until validation scores don't improve for 5000 rounds.
[1000]	training's auc: 0.960071	valid_1's auc: 0.883379
[2000]	training's auc: 0.969076	valid_1's auc: 0.902744
[3000]	training's auc: 0.973976	valid_1's auc: 0.912084
[4000]	training's auc: 0.977145	valid_1's auc: 0.917197
[5000]	training's auc: 0.979403	valid_1's auc: 0.920052
[6000]	training's auc: 0.981273	valid_1's auc: 0.921629
[7000]	training's auc: 0.982932	valid_1's auc: 0.922573
[8000]	training's auc: 0.984452	valid_1's auc: 0.923184
[9000]	training's auc: 0.985871	valid_1's auc: 0.923524
[10000]	training's auc: 0.987194	valid_1's auc: 0.923757
[11000]	training's auc: 0.988447	valid_1's auc: 0.92376
[12000]	training's auc: 0.989606	valid_1's auc: 0.923862
[13000]	training's auc: 0.990673	valid_1's auc: 0.923846
[14000]	training's auc: 0.991655	valid_1's auc: 0.923789
[15000]	training's auc: 0.992586	valid_1's au

In [ ]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":df_test["ID_code"].values})
sub_df["target"] = predictions['target']

In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Data Science Project - Team D/submission/submission_lgbm_fe1_sum.csv", index=False)

## FE2

In [11]:
## Importing data

df_train = pd.read_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe2-sum/train_fe2_sum_lgbm.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Data Science Project - Team D/data/fe2-sum/test_fe2_sum_lgbm.csv")

In [12]:
## Oversampling 

def oversampling(x, y, t=2):
    
    xs, xn = [], []
    
    for i in range(t // 2):
        mask = y == 0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        featnum = x1.shape[1] // 200 - 1

        for c in range(200):
            np.random.shuffle(ids)
            x1[:, [c] + [200 + featnum * c + idc for idc in range(featnum)]] = x1[ids][:, [c] + [200 + featnum * c + idc for idc in range(featnum)]]
        xn.append(x1)
    
    for i in range(t):
        mask = y > 0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        featnum = x1.shape[1] // 200 - 1
        
        for c in range(200):
            np.random.shuffle(ids)
            x1[:, [c] + [200 + featnum * c + idc for idc in range(1)]] = x1[ids][:, [c] + [200 + featnum * c + idc for idc in range(1)]]
        xs.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x, xs, xn])
    y = np.concatenate([y, ys, yn])
    
    return x, y

In [13]:
gbdt_param1 = {
    # Core Parameters
    'objective': 'binary',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 15,
    'tree_learner': 'serial',
    'num_threads': 8,
    'seed': SEED,
    
    # Learning Control Parameters
    'max_depth': -1,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 10,  
    'bagging_fraction': 0.6,
    'bagging_freq': 5,
    'feature_fraction': 0.05,
    'lambda_l1': 1.,
    'bagging_seed': SEED,
    
    # Others
    'verbosity ': 1,
    'boost_from_average': False,
    'metric': 'auc',
}

In [14]:
predictors = df_train.columns.tolist()[2:]
X_test = df_test[predictors]

n_splits = 5
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

predictions = df_test[['ID_code']]

In [ ]:
for fold, (train_ind, val_ind) in enumerate(skf.split(df_train, df_train.target.values)):
    
    X_train, y_train = df_train.iloc[train_ind][predictors], df_train.iloc[train_ind]['target']
    X_valid, y_valid = df_train.iloc[val_ind][predictors], df_train.iloc[val_ind]['target']

    N = 1
    p_valid, yp = 0, 0
        
    for i in range(N):
        print('\nFold {} - N {}'.format(fold + 1, i + 1))
        
        X_t, y_t = oversampling(X_train.values, y_train.values)
        weights = np.array([0.8] * X_t.shape[0])
        weights[:X_train.shape[0]] = 1.0
        print('Shape of X_train after augment: {}\nShape of y_train after augment: {}'.format(X_t.shape, y_t.shape))
        
        X_t = pd.DataFrame(X_t)
        X_t = X_t.add_prefix('var_')
    
        trn_data = lgb.Dataset(X_t, label=y_t, weight=weights)
        val_data = lgb.Dataset(X_valid, label=y_valid)
        evals_result = {}
        
        lgb_clf = lgb.train(gbdt_param1, trn_data, 100000, valid_sets=[trn_data, val_data], early_stopping_rounds=5000, verbose_eval=1000, evals_result=evals_result)
        p_valid += lgb_clf.predict(X_valid)
        yp += lgb_clf.predict(X_test)
        

    predictions['fold{}'.format(fold + 1)] = yp / N


Fold 1 - N 1
Shape of X_train after augment: (336079, 925)
Shape of y_train after augment: (336079,)
Training until validation scores don't improve for 5000 rounds.
[1000]	training's auc: 0.946614	valid_1's auc: 0.881828
[2000]	training's auc: 0.959769	valid_1's auc: 0.899289
[3000]	training's auc: 0.969283	valid_1's auc: 0.908984
[4000]	training's auc: 0.974572	valid_1's auc: 0.914448
[5000]	training's auc: 0.977982	valid_1's auc: 0.917691
[6000]	training's auc: 0.980581	valid_1's auc: 0.919699
[7000]	training's auc: 0.982659	valid_1's auc: 0.920944
[8000]	training's auc: 0.984579	valid_1's auc: 0.921821
[9000]	training's auc: 0.9862	valid_1's auc: 0.922391
[10000]	training's auc: 0.987501	valid_1's auc: 0.922764
[11000]	training's auc: 0.98879	valid_1's auc: 0.92294
[12000]	training's auc: 0.989914	valid_1's auc: 0.923134
[13000]	training's auc: 0.990979	valid_1's auc: 0.923241
[14000]	training's auc: 0.991916	valid_1's auc: 0.923311
[15000]	training's auc: 0.992763	valid_1's auc: 0

In [ ]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)
predictions.to_csv('predictions.csv', index=None)
sub_df = pd.DataFrame({"ID_code":df_test["ID_code"].values})
sub_df["target"] = predictions['target']


In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Data Science Project - Team D/submission/submission_lgbm_fe2_sum.csv", index=False)